# Установим необходимые библиотеки

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings # Для управления предупреждениями

from sklearn import linear_model
# print(sklearn.__version__)
from sklearn.linear_model import LogisticRegression
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,  recall_score, precision_score, roc_curve, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2 , f_classif

from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.feature_selection import RFE
warnings.filterwarnings('ignore')

In [3]:
db_path='C:/Users/nickk/OneDrive/Desktop/TPU/ProgrammingPython/diabetes_coursework/diabetes_012_health_indicators_BRFSS2015.csv'
data= pd.read_csv(db_path)
# Установим максимальное количество отображаемых столбцов
pd.set_option('display.max_columns', None)
data.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


# Выполним преобразование данных в целочисленный тип (integer).

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_012          253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

# Выведем информацию о количестве строк и столбцов датасета

In [5]:
# df.shape
print(f"Num rows: {len(data)}")
print(f"Num columns: {len(data.columns)}")

Num rows: 253680
Num columns: 22


In [6]:
data.shape

(253680, 22)

# Выведем названия столбцов датасета

In [7]:
data.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

# Применим метод isnull().sum(), библиотека pandas, для проверки наличия пропущенных (нулевых) значений в DataFrame data. Этот метод возвращает новый DataFrame того же размера, что и data, где каждое значение заменяется на True, если оно является пропущенным (null или NaN), и на False, если оно не является пропущенным.На выходе получаем логическую матрицу, где True указывает на наличие пропущенного значения.

In [8]:
data.isnull().sum()

Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

In [9]:
# Проверка на наличие хотя бы одного пропущенного значения
has_missing_values = data.isnull().sum().any()
print(has_missing_values)  # Вывод: True

False


# как видим выше, пропущенных значений нет

# Проверим на дубликаты

In [10]:
data.duplicated().sum()

23899

# Метод loc для доступа к строкам и столбцам DF по меткам. В данном случае для выбора строк, которые соответствуют условию, указанному в квадратных скобках, то есть для выбора дубликатов. Двоеточие : после запятой указывает, что мы хотим выбрать все столбцы для этих строк.

In [208]:
data.loc[data.duplicated(),:]

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
1242,2.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,0.0,10.0,4.0,5.0
1563,0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
2700,0.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,6.0,8.0
3160,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
3332,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,6.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253492,2.0,1.0,1.0,1.0,33.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,6.0,6.0
253550,0.0,0.0,0.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,6.0,8.0
253563,0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,6.0,8.0
253597,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,6.0,8.0


# Удалим дубликаты и выведем размер DF после их  удаления.

In [209]:
data.drop_duplicates(inplace=True)
data.shape

(229781, 22)

In [211]:
df_vis=data.copy()

# Выводы: Связь Diabetes_binary с другими столбцами в результате гистограммы:
1: Фрукты, AnyHealthcare, NoDocbccost и пол наименее коррелируют с Diabetes_binary.
2. Высокий уровень АД, холестерина , ИМТ , курение , инсульт, сердечные заболевания или приступы, Физическая активность, Овощи, употребление алкоголя.

# Распределение сахарного диабета среди мужчин и женщин //  Вывод: пол не имеет особого влияния

# Курение

# По-отдельности курение и алкоголь не оказывают сильного влияния на диабет. Проверим их совместную значимость

# Результат: Согласно этим данным, одновременно курение и употребление большого количества алкоголя повышают риск диабета.

# Вероятность развития диабета возрастает по мере того, как у человека возникают сердечные заболевания или приступы

# Инфаркт повышает риск Инсульта. Проверим их совместное влияние на Диабет

# Вывод: инсульт и сердечные заболевания вместе повышают риск развития диабета

# Выводы: диабетики, как правило, имеют высокое кровяное давление и повышенный уровень холестерина. Проверим их совместное влияние на диабет.

# высокий уровень холестерина и высокое кровяное давление тесно связаны друг с другом, поскольку люди с высоким уровнем холестерина, как правило, имеют высокое кровяное давление.
Связь между высоким BP и высоким уровнем холестерина идет в обоих направлениях. Когда организм не может вывести холестерин из кровотока, этот избыток холестерина может откладываться на стенках артерий. Когда артерии становятся жесткими и сужаются из-за отложений, сердцу приходится работать более интенсивно, чтобы прокачивать по ним кровь. Это приводит к тому, что кровяное давление все более повышается.

# Проверка комбинированного влияния HighBP и HighChol на диабет

# Возраст. Поделим по группам

In [238]:
age = pd.cut(df_vis['Age'],bins=[0,4,7,10,12,14],labels=['18:34','35:49','50:64','65:74','75 and older'])
age

0         50:64
1         35:49
2         50:64
3         65:74
4         65:74
          ...  
253675    35:49
253676    65:74
253677    18:34
253678    35:49
253679    50:64
Name: Age, Length: 229781, dtype: category
Categories (5, object): ['18:34' < '35:49' < '50:64' < '65:74' < '75 and older']

# Люди в возрасте от 50 до 64 лет имеют более высокий риск развития диабета

# PhysHlth

# MentHlth

# Психическое здоровье не влияет на диабет

# Общее здоровье оказывает влияние на диабет

# Income ( не оказывает)

# Education Образование

# Veggies

# Fruits

# PhysActivity

# Вывод: физическая активность влияет на общее здоровье, поскольку люди, занимающиеся физическими упражнениями, имеют намного лучшее общее состояние здоровья

# Итоги по визуализации:
1: мужчины и женщины одинаково уязвимы для диабета.
2: люди старше 45 лет более уязвимы для диабета, чем молодые.с увеличением возраста увеличивается и число людей, страдающих диабетом.
3. Более половины диабетиков страдают ожирением, почти половина преддиабетиков страдают ожирением.
4. процент диабетиков и преддиабетиков, страдающих ожирением и избыточным весом, намного выше, чем процент недиабетиков, страдающих ожирением и избыточным весом
5: когда уровень образования повышается, число людей, страдающих диабетом, сокращается.
6: у людей с более низким доходом риск развития диабета выше, чем у людей с более высоким доходом.
7: питание оказывает влияние (потребление овощей и фруктов снижает риск диабета)
8: Физическая активность снижает риск развития диабета.


# Preprocessing // Предварительная подготовка данных

# Рассмотрим выбросы по BMI

# Исключим аномалии (-1)

In [263]:
df[df['anomaly']==-1] # выберем аномалии (каждый раз их колич.меняется)

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,anomaly
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0,-1
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0,-1
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0,-1
6,0.0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0,-1
8,2.0,1.0,1.0,1.0,30.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253670,2.0,1.0,1.0,1.0,25.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,5.0,15.0,0.0,1.0,0.0,13.0,6.0,4.0,-1
253671,0.0,1.0,1.0,1.0,23.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,0.0,5.0,0.0,1.0,8.0,3.0,2.0,-1
253672,0.0,1.0,0.0,1.0,30.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,2.0,1.0,-1
253676,2.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0,-1


In [264]:
df[df['anomaly']==-1].shape

(80892, 23)

In [265]:
df.drop(df[df['anomaly']==-1].index,inplace = True)

In [266]:
df.shape

(148290, 23)

In [267]:
df

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,anomaly
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0,1
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0,1
5,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0,1
7,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0,1
9,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253673,0.0,1.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,14.0,4.0,0.0,1.0,3.0,6.0,8.0,1
253674,0.0,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,6.0,5.0,1
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0,1
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,1


# Drop column(anomaly)

In [268]:
df.drop(columns=['anomaly'], inplace=True)

In [269]:
df.shape

(148290, 22)

# Масштабирование данных (Scaling data)

In [270]:
x = df.drop(['Diabetes_012'],axis=1)
y = df['Diabetes_012']

In [271]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)

StandardScaler()

In [272]:
scaled_features = scaler.transform(x)
x = pd.DataFrame(scaled_features,columns=df.columns[1:])
x.head(10)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.292291,-0.798139,0.156633,-0.116261,-0.827866,-0.08887,-0.175543,0.446280,0.699300,0.388136,-0.19509,0.113347,-0.158632,-0.284017,-0.361135,-0.371090,-0.210098,-0.871500,1.002092,-2.457427,-0.285176
1,1.292291,1.252914,0.156633,-0.688968,-0.827866,-0.08887,-0.175543,0.446280,0.699300,0.388136,-0.19509,0.113347,-0.158632,-0.284017,0.336987,-0.371090,-0.210098,-0.871500,1.002092,-0.197326,-1.450668
2,1.292291,1.252914,0.156633,-0.498066,1.207925,-0.08887,-0.175543,0.446280,0.699300,0.388136,-0.19509,0.113347,-0.158632,-0.284017,-0.361135,0.078313,-0.210098,1.147446,0.676794,0.932724,0.880316
3,1.292291,1.252914,0.156633,-0.498066,1.207925,-0.08887,-0.175543,0.446280,-1.430002,0.388136,-0.19509,0.113347,-0.158632,0.913779,-0.361135,-0.371090,4.759673,-0.871500,1.002092,-1.327377,-1.450668
4,-0.773820,-0.798139,0.156633,-0.688968,-0.827866,-0.08887,-0.175543,-2.240748,-1.430002,0.388136,-0.19509,0.113347,-0.158632,-0.284017,-0.361135,-0.371090,-0.210098,1.147446,0.026199,-1.327377,-2.033414
5,-0.773820,-0.798139,0.156633,-0.498066,1.207925,-0.08887,-0.175543,0.446280,0.699300,0.388136,-0.19509,0.113347,-0.158632,0.913779,-0.361135,-0.371090,-0.210098,1.147446,1.652687,0.932724,0.880316
6,-0.773820,-0.798139,0.156633,-0.307163,1.207925,-0.08887,-0.175543,-2.240748,-1.430002,0.388136,-0.19509,0.113347,-0.158632,0.913779,-0.361135,2.999428,-0.210098,-0.871500,-0.299099,-0.197326,0.297570
7,1.292291,-0.798139,0.156633,1.029152,-0.827866,-0.08887,-0.175543,0.446280,-1.430002,-2.576414,-0.19509,0.113347,-0.158632,-0.284017,0.802401,-0.371090,-0.210098,-0.871500,-0.624396,0.932724,0.880316
8,1.292291,1.252914,0.156633,-1.261675,-0.827866,-0.08887,-0.175543,0.446280,0.699300,0.388136,-0.19509,0.113347,-0.158632,0.913779,-0.361135,-0.371090,-0.210098,-0.871500,0.676794,-1.327377,-2.033414
9,-0.773820,-0.798139,0.156633,-0.879870,1.207925,-0.08887,-0.175543,0.446280,-1.430002,-2.576414,-0.19509,0.113347,-0.158632,-0.284017,-0.361135,-0.371090,-0.210098,1.147446,-0.299099,-0.197326,-0.285176


In [273]:
# Разделим данные (Split the data)

In [274]:
x_train , x_test , y_train , y_test = train_test_split(x,y , test_size=0.35, random_state=0, shuffle =True)

In [278]:
df['Diabetes_012'].value_counts()

Diabetes_012
0.0    135333
2.0     10778
1.0      2179
Name: count, dtype: int64

# Вывод: Из-за несбалансированности данных точность определения больных диабетом низкая, поэтому используем SMOTE + ENN SMOTE --> Метод синтетической передискретизации меньшинства SMOTE + ENN - это метод, в котором больше нет. количество наблюдений удаляется из пространства выборки. ENN - это метод занижения выборки, при котором оцениваются ближайшие соседи каждого из мажоритарного класса. Если ближайшие соседи неправильно классифицируют этот конкретный экземпляр мажоритарного класса, то этот экземпляр удаляется. Интеграция этого метода с данными с избыточной выборкой, выполненная SMOTE, помогает выполнить обширную очистку данных. Здесь при неправильной классификации NN образцы из обоих классов удалены. Это приводит к более четкому и сжатому разделению классов.

# Повторная выборка // Resampling

In [281]:
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
x_resampled, y_resampled = sm.fit_resample(x,y)

In [282]:
xre_train,xre_test,yre_train,yre_test = train_test_split(x_resampled, y_resampled, test_size=0.3, random_state=42)

# Modeling

# Балансировка классов с помощью SMOTE
smote = SMOTE()
xre_train, yre_train = smote.fit_resample(xre_train, yre_train)
print(pd.Series(yre_train).value_counts())

In [348]:
# Выполняем кросс-валидацию
cv_scores = cross_val_score(lg, xre_train, yre_train, cv=5)  # cv=5 означает 5 фолдов
print("Cross-validation scores:", cv_scores)

Cross-validation scores: [0.56858064 0.57078558 0.57144159 0.57309206 0.57475035]


In [349]:
lg.fit(xre_train, yre_train)  # Обучаем модель


LogisticRegression(class_weight='balanced', penalty='l1', random_state=0,
                   solver='liblinear')

In [350]:
print(lg.score(xre_train, yre_train))
print(lg.score(xre_test, yre_test))

0.5718029178192526
0.5492045928816158


In [351]:
y_pred_train_lg = lg.predict(xre_train)
acc_train_lg = accuracy_score(yre_train, y_pred_train_lg)

y_pred_test_lg = lg.predict(xre_test)
acc_test_lg = accuracy_score(yre_test, y_pred_test_lg)
print(acc_train_lg)
print(acc_test_lg)

# Расчет ошибок
print(f'MSE: { mean_squared_error(yre_test, y_pred_test_lg)}')
print(f'MAE: { mean_absolute_error(yre_test, y_pred_test_lg)}')

print(classification_report(yre_test, y_pred_test_lg))

0.5718029178192526
0.5492045928816158
MSE: 0.6665599006105077
MAE: 0.522716904949092
              precision    recall  f1-score   support

         0.0       0.60      0.76      0.67     28272
         1.0       0.51      0.28      0.36     39392
         2.0       0.53      0.68      0.59     35365

    accuracy                           0.55    103029
   macro avg       0.55      0.57      0.54    103029
weighted avg       0.54      0.55      0.53    103029



In [352]:
y_pred_prob_lg = lg.predict_proba(xre_test)
roc_auc_score_lg = roc_auc_score(yre_test, y_pred_prob_lg, multi_class="ovr", average='macro')
print('ROC AUC Score:', roc_auc_score_lg) 

ROC AUC Score: 0.747720555241226


# Вычисление корреляции на полных данных
correlation = df.corr()

# Удаление признаков с корреляцией меньше порога (например, 0.1)
threshold = 0.1
low_correlation_features = correlation[correlation['Diabetes_012'].abs() < threshold].index.tolist()
print('Низкокоррелирующие признаки:', low_correlation_features)

'''
# Проверяем, какие из низкокоррелирующих признаков есть в xre_train
existing_features = [feature for feature in low_correlation_features if feature in xre_train.columns]
print("Признаки, которые существуют в xre_train:", existing_features)

# Удаление признаков из xre_train
xre_train = xre_train.drop(columns=existing_features)
'''


# Стандартизация данных
scaler = StandardScaler()
xre_train = scaler.fit_transform(xre_train)

# Методы отбора признаков
selector = SelectKBest(score_func=f_classif, k='all')  # Выбор всех признаков
print(selector)
xre_train = selector.fit_transform(xre_train, yre_train)

# Выполняем кросс-валидацию
cv_scores = cross_val_score(lg, xre_train, yre_train, cv=5)  # cv=5 означает 5 фолдов
print("Cross-validation scores:", cv_scores)

lg.fit(xre_train, yre_train)  # Обучаем модель

print(lg.score(xre_train, yre_train))
print(lg.score(xre_test, yre_test))

y_pred_train_lg = lg.predict(xre_train)
acc_train_lg = accuracy_score(yre_train, y_pred_train_lg)

y_pred_test_lg = lg.predict(xre_test)
acc_test_lg = accuracy_score(yre_test, y_pred_test_lg)
print(acc_train_lg)
print(acc_test_lg)

# Расчет ошибок
mse = mean_squared_error(yre_test, y_pred_test_lg)
mae = mean_absolute_error(yre_test, y_pred_test_lg)

print(classification_report(yre_test, y_pred_test_lg))

print('Precision: %.3f' % precision_score(yre_test, y_pred_test_lg, average="micro"))
print('Recall: %.3f' % recall_score(yre_test, y_pred_test_lg, average="micro"))
print('F-measure: %.3f' % f1_score(yre_test, y_pred_test_lg, average="micro"))

y_pred_prob_lg = lg.predict_proba(xre_test)
roc_auc_score_lg = roc_auc_score(yre_test, y_pred_prob_lg, multi_class="ovr", average='macro')
print('ROC AUC Score:', roc_auc_score_lg) 

#  LogisticRegression - 1 Version

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Разделяем данные на классы
class_0 = df[df['Diabetes_012'] == 0]
class_1 = df[df['Diabetes_012'] == 1]
class_2 = df[df['Diabetes_012'] == 2]

# Устанавливаем количество образцов для меньших классов
n_samples = min(len(class_0), len(class_1), len(class_2))  # Количество образцов для класса 1 и 2

# Уменьшаем класс 0 до n_samples
class_0_downsampled = class_0.sample(n=n_samples, random_state=100)

# Объединяем классы обратно
df_balanced = pd.concat([class_0_downsampled, class_1, class_2])

# Перемешиваем данные
df_balanced = df_balanced.sample(frac=1, random_state=100).reset_index(drop=True)

# Проверяем размерности классов
print(df_balanced['Diabetes_012'].value_counts())

# Удаляем ненужные столбцы
columns_to_drop = ['Fruits', 'Sex', 'NoDocbcCost','CholCheck', 'AnyHealthcare']
df_filtered = df_balanced.drop(columns=columns_to_drop)

# Определяем целевую переменную и признаки
X = df_filtered.drop('Diabetes_012', axis=1)  # 
y = df_filtered['Diabetes_012']

# Создаем модель
model_lg = LogisticRegression(random_state=100, solver='liblinear', class_weight='balanced')

# Выполняем кросс-валидацию
cv_scores = cross_val_score(model, X, y, cv=5)  # cv=5 означает 5 фолдов


# Обучаем модель
model_lg.fit(X_train, y_train)

# Предсказываем на тестовой выборке
y_pred = model_lg.predict(X_test)

# Расчет ошибок
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Выводим результаты
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Отчет о классификации
print(classification_report(y_test, y_pred))

# Выводим результаты
print("Cross-validation scores:", cv_scores)
print("Mean CV score:", cv_scores.mean())
print("Standard deviation of CV score:", cv_scores.std())


# Определяем целевую переменную и признаки
#X = df_filtered.drop('Diabetes_012', axis=1)
#y = df_filtered['Diabetes_012']

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100, stratify=y)

# Создаем модель
model_lg = LogisticRegression(random_state=100, solver='liblinear', class_weight='balanced')

# Обучаем модель
model_lg.fit(X_train, y_train)

# Предсказываем на тестовой выборке
y_pred = model_lg.predict(X_test)

# Расчет ошибок
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Выводим результаты
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Отчет о классификации
print(classification_report(y_test, y_pred))

# Визуализация: матрица путаницы
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# model.fit(X, y)  # Обучите модель на всем наборе данных
y_pred = model.predict(X)
print(classification_report(y, y_pred))

# Logistic Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import (classification_report, accuracy_score, 
                             mean_squared_error, mean_absolute_error, 
                             precision_score, recall_score, f1_score, 
                             roc_auc_score)
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

print(df.columns)
# OUTPUT Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker','Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
#       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education','Income'], dtype='object')

# Вычисление корреляции
correlation = df.corr()

# Удаление признаков с корреляцией меньше порога (например, 0.1)
threshold = 0.1
low_correlation_features = correlation[correlation['Diabetes_012'].abs() < threshold].index.tolist()

# Удаление признаков
df_reduced = df.drop(columns=low_correlation_features)

print("Оставшиеся признаки после удаления с низкой корреляцией:")
#print(df_reduced.columns)
# OUTPUT Оставшиеся признаки после удаления с низкой корреляцией:
# Index(['Diabetes_012', 'HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age'], dtype='object')


# Разделение на признаки и целевую переменную
X = df_reduced.drop(columns=['Diabetes_012'])  # Признаки
y = df_reduced['Diabetes_012']  # Целевая переменная

# Балансировка классов с помощью SMOTE
smote = SMOTE()
X, y = smote.fit_resample(X, y)

# Стандартизация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Методы отбора признаков
selector = SelectKBest(score_func=f_classif, k='all')  # Выбор всех признаков
X_selected = selector.fit_transform(X_scaled, y)

# Обучение модели с кросс-валидацией
model = LogisticRegression(max_iter=200)

# Кросс-валидация
scores = cross_val_score(model, X_selected, y, cv=5)  # 5-кратная кросс-валидация
print("Кросс-валидация, точности для каждого фолда:", scores)
print("Средняя точность:", scores.mean())

# Обучение модели на отобранных признаках
model.fit(X_selected, y)

# Предсказание и оценка на тестовом наборе (в этом примере мы используем те же данные)
y_pred = model.predict(X_selected)

# Вывод результатов
print("Отчет о классификации:")
print(classification_report(y, y_pred))
print("Точность модели:", accuracy_score(y, y_pred))

# Балансировка классов с помощью SMOTE (еще раз??)
smote = SMOTE()
xre_train, yre_train = smote.fit_resample(xre_train, yre_train)

# Подбор гиперпараметров
param_grid = {'C': [0.01, 0.1, 1.0,10,100], 'penalty': ['l1']}
grid_search = GridSearchCV(LogisticRegression(solver='saga', multi_class='ovr'), 
                           param_grid, cv=5, error_score='raise')
grid_search.fit(xre_train, yre_train)
print('The best parameters: ', grid_search.best_params_)
# Output: The best parameters:  {'C': 0.01, 'penalty': 'l1'}

# Проверка после SMOTE
print(f'Размеры после SMOTE: {xre_train.shape[0]}, {yre_train.shape[0]}')

print(xre_train.columns)

xre_train = xre_train.drop(columns=['HighChol'])
xre_test = xre_test.drop(columns=['HighChol'])

print(xre_train.columns)

xre_train = xre_train.drop(columns=['PhysActivity'])
xre_test = xre_test.drop(columns=['PhysActivity'])

# Создание и обучение модели с выбранными лучшими параметрами
lg = LogisticRegression(C=0.01, penalty='l1', solver='saga', multi_class='multinomial', max_iter=500)
lg.fit(xre_train, yre_train)

# Кросс-валидация
scores = cross_val_score(lg, xre_train, yre_train, cv=5)  # cv - количество фолдов

# Вывод результатов
print("Кросс-валидация, точности для каждого фолда:", scores)
print("Средняя точность:", scores.mean())

correlation = df.corr()
print(correlation['Diabetes_012'].abs().sort_values(ascending=False))

# Предсказания

y_pred_train_lg = lg.predict(xre_train)
y_pred_test_lg = lg.predict(xre_test)

# Оценка модели
print('Training set score: {:.4f}'.format(lg.score(xre_train, yre_train)))
print('Test set score: {:.4f}'.format(lg.score(xre_test, yre_test)))
print('Accuracy on training set: {:.4f}'.format(accuracy_score(yre_train, y_pred_train_lg)))
print('Accuracy on test set: {:.4f}'.format(accuracy_score(yre_test, y_pred_test_lg)))

# Метрики ошибок
mse_lg = mean_squared_error(yre_test, y_pred_test_lg)
mae_lg = mean_absolute_error(yre_test, y_pred_test_lg)
print('MSE_lg: {:.3f}'.format(mse_lg))
print('MAE_lg: {:.3f}'.format(mae_lg))

# Классификационный отчет
print(classification_report(yre_test, y_pred_test_lg))
print('Precision: {:.3f}'.format(precision_score(yre_test, y_pred_test_lg, average="micro")))
print('Recall: {:.3f}'.format(recall_score(yre_test, y_pred_test_lg, average="micro")))
print('F-measure: {:.3f}'.format(f1_score(yre_test, y_pred_test_lg, average="micro")))

# ROC AUC Score
y_pred_prob_lg = lg.predict_proba(xre_test)
roc_auc_score_lg = roc_auc_score(yre_test, y_pred_prob_lg, multi_class="ovr")
print('ROC AUC Score:', roc_auc_score_lg)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(xre_train.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Корреляционная матрица признаков")
plt.show()

# print(xre_train.corr())